In [ ]:
import pprint
def readlog(logs):
    log_dict = {}
    for log in logs:
        values = log.split()
        date = values[0]
        time = values[1]
        activity = values[2]
        server = values[3]
        email = values[4]
        
         
        if email in log_dict:
            if date in log_dict[email]:
                log_dict[email][date].append((time, activity, server))
            else:
                log_dict[email][date] = [(time, activity, server)]
        else:
            log_dict[email] = {date:[(time,activity, server)]}
        log_dict[email][date].sort()
    pprint.pprint(log_dict)
    return log_dict

In [ ]:
# Will write reports for each main report besides domain count (suspicious_acts, irresponsible_acts, system_glitch). We found
# that the domain count report was easier done by itself.
def writeReport(title, filename, acts, counts, log_dict):
    new_file = open(filename, "w")
    new_file.write("===============================\n")
    new_file.write(f"==={title}({str(counts)}cases) ===\n")
    new_file.write("=================================\n")
    
    for email in acts:
        act_count = len(acts[email])
        new_file.write(f"{email}\t\t{str(act_count)}\n")
        
        for date in acts[email]:
            new_file.write(f"\tDATE:[{date}]---\n")
            
            for activity in log_dict[email][date]:
                new_file.write(f"\t\t{activity[0]}\t{activity[1]}\t{activity[2]}\n")
    new_file.close()
        

In [ ]:
userlog = open("userlog.log")
logs = userlog.readlines()
logs_dict= readlog(logs) 

In [ ]:
suspicious_acts= {}
suspicious_count = 0
for email in log_dict:
    for date in log_dict[email]:
        for values in log_dict[email][date]:
            i = 0
            hour = int(values[0].split(':')[0])
            if values[1] == "login":
                i = i + 1
                if i > 5:
                    suspicious_count += 1
                    dates = suspicious_acts.get(email,[])
                    dates.append(date)
                    suspicious_acts[email] = dates
                    
                         
            
            if values[1] == "login" and hour < 5:
                suspicious_count +=1
                dates = suspicious_acts.get(email,[])
                dates.append(date)
                suspicious_acts[email] = dates
                    
                    
                    
                suspicious_acts[email].sort()

print(suspicious_acts)
writeReport("suspicious activities", "suspicious_report.txt", suspicious_acts, suspicious_count, log_dict)         

In [ ]:
irresponsible_acts = {}
irresponsible_count = 0
for email in log_dict:
    for date in log_dict[email]:
        login = 0
        logout = 0
        for values in log_dict[email][date]:
            if values[1] == 'login':
                login = login+1
            if values[1] == 'logout':
                logout = logout + 1
                    
                if login > logout:
                    irresponsible_count +=1
                    dates = irresponsible_acts.get(email,[])
                    dates.append(date)
                    irresponsible_acts[email] = dates     
                     
                    irresponsible_acts[email].sort()  
                    
        
print(irresponsible_acts)    
writeReport("irresponsible activities", "irresponsible_report.txt", irresponsible_acts, irresponsible_count, log_dict)         

In [ ]:
# Initializes logins and logouts as 0 and uses the irresponsible_acts as its dicitonary
    logins = 0
    logouts = 0
    irresponsible_acts = {}
    glitch_count = 0
    
# It goes through dates in the main log_dict and checks each activity for a login. If there is a login it adds 1
# otherwise it adds 1 to the logouts. If logins < logouts it counts as a system glitch.
def system_glitch():
    for date in log_dict:
        irresponsible_acts[date] = {}
        for email in log_dict[date]:
            logins = 0
            logouts = 0
            for session in log_dict[date][email]:
                if session['activity'] == 'login':
                    logins += 1
                else:
                    logouts += 1
            if logins < logouts:
                glitch_count += 1
    return irresponsible_acts

print(system_glitch)

# Write report 
writeReport("System Glitch", "glitch_report.txt", irresponsible_acts, glitch_count, log_dict)

In [ ]:
def domain_count():
    file_name = "domain_report.txt"
    log_dict= {}

# For each email in the log, it will split the email at the @ symbol leaving the domain and if there are more than 1 usernames
# for a single domain, it will add 1 to the count (using the loop). This code will count the total usernames under a single
# domain and display that value.
    for email in log_dict:
        domain_name = email.split("@")
        if str(domain_name) in log_dict:
            log_dict[str(domain_name)] += 1
        else:
            log_dict[str(domain_name)] = 1
            
            
# Create report with title, "N" domains as seen in the provided example
    f = open(file_name, "w")
    f.write("    ====================\n")
    f.write("=== DOMAIN COUNT " + "("+str(len(log_dict)) +" domains)"+ " ===\n")
    f.write("    ====================\n")
    
# This prints the number of usernames belong to a specific domain
    for domain,count in log_dict.items():
        f.write(domain+":"+"  "+str(count)+"\n")
    f.close()

In [ ]:
domain_count()
print(open("domain_report.txt","r").read())